<a href="https://colab.research.google.com/github/jadenfix/supervised_fine_tuning/blob/main/another_coding_llm_more_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Force-reinstall a clean, matching torch + transformers
!pip uninstall -y torch torchvision torchaudio transformers accelerate bitsandbytes trl peft
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers datasets accelerate peft bitsandbytes trl

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
# 1) Installs (after reset)
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers datasets accelerate peft bitsandbytes trl

# 2) Imports & config
import os
import torch

# Ensure it’s the real torch
print("Torch version:", torch.__version__)
print("Is CUDA available?", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0))

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# 3) Colab T4 sanity check
assert torch.cuda.is_available() and "T4" in torch.cuda.get_device_name(0)

# 4) Dataset
dataset = load_dataset("codeparrot/github-code", split="train[:2000]")

# 5) 8-bit quantization config
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/CodeLlama-7b-Instruct-hf",
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-7b-Instruct-hf")
tokenizer.pad_token = tokenizer.eos_token

# 6) LoRA
peft_config = LoraConfig(
    r=8, lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# 7) Training args
training_args = TrainingArguments(
    output_dir="codellama-qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    learning_rate=2e-4,
    max_steps=200,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

# 8) Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="content",
    args=training_args,
    peft_config=peft_config,
    max_seq_length=256
)

# 9) Kick off
trainer.train()

Looking in indexes: https://download.pytorch.org/whl/cu124
Torch version: 2.6.0+cu124
Is CUDA available? True
GPU device: Tesla T4


README.md:   0%|          | 0.00/7.54k [00:00<?, ?B/s]

github-code.py:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

The repository for codeparrot/github-code contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/github-code.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-01126.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

train-00195-of-01126.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

train-00196-of-01126.parquet:   0%|          | 0.00/288M [00:00<?, ?B/s]

train-00197-of-01126.parquet:   0%|          | 0.00/288M [00:00<?, ?B/s]

train-00198-of-01126.parquet:   0%|          | 0.00/293M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import BitsAndBytesConfig

# Load dataset (e.g., Alpaca-style)
dataset = load_dataset("tatsu-lab/alpaca", split="train[:2000]")

# Load TinyLlama with 8-bit and PEFT
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# LoRA Config (optional, but helps on T4)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="tinyllama-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    fp16=True,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

# SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    args=training_args,
    peft_config=peft_config,
    max_seq_length=512,
)

# Train
trainer.train()

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
partially initialized module 'torch._dynamo' has no attribute 'config' (most likely due to a circular import)